In [1]:
import poker_environment as pe_
from poker_environment import AGENT_ACTIONS, BETTING_ACTIONS
import copy
import random

"""
Player class
"""
class PokerPlayer(object):
    def __init__(self, current_hand_=None, stack_=300, action_=None, action_value_=None):
        self.current_hand = current_hand_
        self.current_hand_type = []
        self.current_hand_strength = []
        self.stack = stack_
        self.action = action_
        self.action_value = action_value_

    """
    identify agent hand and evaluate it's strength
    """
    def evaluate_hand(self):
        self.current_hand_type = pe_.identify_hand(self.current_hand)
        self.current_hand_strength = pe_.Types[self.current_hand_type[0]]*len(pe_.Ranks) + pe_.Ranks[self.current_hand_type[1]]

    """
    return possible actions, fold if there is not enough money...
    """
    def get_actions(self):
        actions_ = []
        for _action_ in AGENT_ACTIONS:
            if _action_[:3] == 'BET' and int(_action_[3:])>=(self.stack):
                actions_.append('FOLD')
            else:
                actions_.append(_action_)
        return set(actions_)

"""
Game State class
"""
class GameState(object):
    def __init__(self,
                 nn_current_hand_=None,
                 nn_current_bidding_=None,
                 phase_ = None,
                 pot_=None,
                 acting_agent_=None,
                 parent_state_=None,
                 children_state_=None,
                 agent_=None,
                 opponent_=None):

        self.nn_current_hand = nn_current_hand_
        self.nn_current_bidding = nn_current_bidding_
        self.phase = phase_
        self.pot = pot_
        self.acting_agent = acting_agent_
        self.parent_state = parent_state_
        self.children = children_state_
        self.agent = agent_
        self.opponent = opponent_
        self.showdown_info = None

    """
    draw 10 cards randomly from a deck
    """
    def dealing_cards(self):
        agent_hand, opponent_hand = pe_.generate_2hands()
        self.agent.current_hand = agent_hand
        self.agent.evaluate_hand()
        self.opponent.current_hand = opponent_hand
        self.opponent.evaluate_hand()
        #self.showdown_info = None

    """
    draw 10 cards from a fixed sequence of hands
    """
    def dealing_cards_fixed(self):
        self.agent.current_hand = pe_.fixed_hands[self.nn_current_hand][0]
        self.agent.evaluate_hand()
        self.opponent.current_hand = pe_.fixed_hands[self.nn_current_hand][1]
        self.opponent.evaluate_hand()

    """
    SHOWDOWN phase, assign pot to players
    """
    def showdown(self):

        if self.agent.current_hand_strength == self.opponent.current_hand_strength:
            self.showdown_info = 'draw'
            if self.acting_agent == 'agent':
                self.agent.stack += (self.pot - 5) / 2. + 5
                self.opponent.stack += (self.pot - 5) / 2.
            else:
                self.agent.stack += (self.pot - 5) / 2.
                self.opponent.stack += (self.pot - 5) / 2. + 5
        elif self.agent.current_hand_strength > self.opponent.current_hand_strength:
            self.showdown_info = 'agent win'
            self.agent.stack += self.pot
        else:
            self.showdown_info = 'opponent win'
            self.opponent.stack += self.pot

    # print out necessary information of this game state
    def print_state_info(self):

        print('************* state info **************')
        print('nn_current_hand', self.nn_current_hand)
        print('nn_current_bidding', self.nn_current_bidding)
        print('phase', self.phase)
        print('pot', self.pot)
        print('acting_agent', self.acting_agent)
        print('parent_state', self.parent_state)
        print('children', self.children)
        print('agent', self.agent)
        print('opponent', self.opponent)

        if self.phase == 'SHOWDOWN':
            print('---------- showdown ----------')
            print('agent.current_hand', self.agent.current_hand)
            print(self.agent.current_hand_type, self.agent.current_hand_strength)
            print('opponent.current_hand', self.opponent.current_hand)
            print(self.opponent.current_hand_type, self.opponent.current_hand_strength)
            print('showdown_info', self.showdown_info)

        print('----- agent -----')
        print('agent.current_hand',self.agent.current_hand)
        print('agent.current_hand_type',self.agent.current_hand_type)
        print('agent.current_hand_strength',self.agent.current_hand_strength)
        print('agent.stack',self.agent.stack)
        print('agent.action',self.agent.action)
        print('agent.action_value',self.agent.action_value)

        print('----- opponent -----')
        print('opponent.current_hand', self.opponent.current_hand)
        print('opponent.current_hand_type',self.opponent.current_hand_type)
        print('opponent.current_hand_strength',self.opponent.current_hand_strength)
        print('opponent.stack',self.opponent.stack)
        print('opponent.action',self.opponent.action)
        print('opponent.action_value',self.opponent.action_value)
        print('**************** end ******************')

# copy given state in the argument
def copy_state(game_state):
    _state = copy.copy(game_state)
    _state.agent = copy.copy(game_state.agent)
    _state.opponent = copy.copy(game_state.opponent)
    return _state


# Use the function when optimum path is found to print the states
def print_path(state):
    nn_level = 0
    while state.parent_state != None:
        nn_level += 1
        print(nn_level)
        state.print_state_info()
        state = state.parent_state

"""
successor function for generating next state(s)
"""
def get_next_states(last_state):

    if last_state.phase == 'SHOWDOWN' or last_state.acting_agent == 'opponent' or last_state.phase == 'INIT_DEALING':

        # NEW BETTING ROUND, AGENT ACT FIRST

        states = []

        for _action_ in last_state.agent.get_actions():

            _state_ = copy_state(last_state)
            _state_.acting_agent = 'agent'

            if last_state.phase == 'SHOWDOWN' or last_state.phase == 'INIT_DEALING':
                _state_.dealing_cards_fixed()

            if _action_ == 'CALL':

                _state_.phase = 'SHOWDOWN'
                _state_.agent.action = _action_
                _state_.agent.action_value = 5
                _state_.agent.stack -= 5
                _state_.pot += 5

                _state_.showdown()

                _state_.nn_current_hand += 1
                _state_.nn_current_bidding = 0
                _state_.pot = 0
                _state_.parent_state = last_state
                states.append(_state_)

            elif _action_ == 'FOLD':

                _state_.phase = 'SHOWDOWN'
                _state_.agent.action = _action_
                _state_.opponent.stack += _state_.pot

                _state_.nn_current_hand += 1
                _state_.nn_current_bidding = 0
                _state_.pot = 0
                _state_.parent_state = last_state
                states.append(_state_)


            elif _action_ in BETTING_ACTIONS:

                _state_.phase = 'BIDDING'
                _state_.agent.action = _action_
                _state_.agent.action_value = int(_action_[3:])
                _state_.agent.stack -= int(_action_[3:])
                _state_.pot += int(_action_[3:])

                _state_.nn_current_bidding += 1
                _state_.parent_state = last_state
                states.append(_state_)

            else:

                print('...unknown action...')
                exit()
        last_state.children = states
        return states

    elif last_state.phase == 'BIDDING' and last_state.acting_agent == 'agent':

        states = []
        _state_ = copy_state(last_state)
        _state_.acting_agent = 'opponent'

        opponent_action, opponent_action_value = pe_.poker_strategy_example(last_state.opponent.current_hand_type[0],
                                                                            last_state.opponent.current_hand_type[1],
                                                                            last_state.opponent.stack,
                                                                            last_state.agent.action,
                                                                            last_state.agent.action_value,
                                                                            last_state.agent.stack,
                                                                            last_state.pot,
                                                                            last_state.nn_current_bidding)
        print()
        if opponent_action =='CALL':

            _state_.phase = 'SHOWDOWN'
            _state_.opponent.action = opponent_action
            _state_.opponent.action_value = 5
            _state_.opponent.stack -= 5
            _state_.pot += 5

            _state_.showdown()

            _state_.nn_current_hand += 1
            _state_.nn_current_bidding = 0
            _state_.pot = 0
            _state_.parent_state = last_state
            states.append(_state_)

        elif opponent_action == 'FOLD':

            _state_.phase = 'SHOWDOWN'

            _state_.opponent.action = opponent_action
            _state_.agent.stack += _state_.pot

            _state_.nn_current_hand += 1
            _state_.nn_current_bidding = 0
            _state_.pot = 0
            _state_.parent_state = last_state
            states.append(_state_)

        elif opponent_action + str(opponent_action_value) in BETTING_ACTIONS:

            _state_.phase = 'BIDDING'

            _state_.opponent.action = opponent_action
            _state_.opponent.action_value = opponent_action_value
            _state_.opponent.stack -= opponent_action_value
            _state_.pot += opponent_action_value

            _state_.nn_current_bidding += 1
            _state_.parent_state = last_state
            states.append(_state_)

        else:
            print('unknown_action')
            exit()
            
        last_state.children = states
        return states



############################################################################################################
"""
Game flow:
Two agents will keep playing until one of them lose 100 coins or more.
"""
#############################################################################################################

MAX_HANDS = 4
INIT_AGENT_STACK = 400

# initialize 2 agents and a game_state
agent = PokerPlayer(current_hand_=None, stack_=INIT_AGENT_STACK, action_=None, action_value_=None)
opponent = PokerPlayer(current_hand_=None, stack_=INIT_AGENT_STACK, action_=None, action_value_=None)


init_state = GameState(nn_current_hand_=0,
                       nn_current_bidding_=0,
                       phase_ = 'INIT_DEALING',
                       pot_=0,
                       acting_agent_=None,
                       agent_=agent,
                       opponent_=opponent,
                       )


game_state_queue = []
game_on = True
round_init = True
count=0
Root=[]
exp_nodes=[]
steps = 10000

################################### GAME START #####################################################
############## Building a Tree, Implementing Search Algorithm, BFS ########################################
while game_on:

    if round_init:
        round_init = False
        states_ = get_next_states(init_state)
        game_state_queue.extend(states_[:])
        for i in range(len(states_)):
            Root.append(states_[i])
            exp_nodes.append(states_[i])

    else:
        Temp = []
        for i in range(len(states_)):
            if (states_[i].nn_current_hand <= 4):
                Temp.extend(get_next_states(states_[i]))

        states_ = Temp[:]
        game_state_queue.extend(states_[:])


        for _state_ in states_:
            exp_nodes.append(_state_)
            if (_state_.nn_current_hand <= 4) and (_state_.phase == 'SHOWDOWN') and (_state_.agent.stack >= 500):
                #bb= Tree[ii]
                end_state_ = _state_
                #print(_state_)
                Len_BFS = len(exp_nodes)
                print("Length of Explored Nodes BFS: ", Len_BFS)
                game_on = False
                break

print('######################### Path found using BFS algorithm #########################################')


{'BET10', 'CALL', 'BET5', 'BET25', 'FOLD'}
{'BET10', 'BET5', 'BET25'}
{'BIDDING', 'SHOWDOWN', 'INIT_DEALING'}
opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: me

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

...

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent_hand_strength: median

opponent

In [2]:
######################################### DFS ###############################################333
def PokerGame_DFSSearch(Root,steps,type):
    open = []
    closed = []
    check = 0
    exp_nodes=[]
    for i in range(len(Root)):
        if check==1:
            break
        node = Root[i]
        open.append(node)
        while len(open) > 0:
            current_node = open.pop(-1)
            closed.append(current_node)
            if current_node.children is not None:
                for adj_ in current_node.children:
               # open.append(adjacent_vertex)
                    if (adj_ in closed):
                        continue
                    if (adj_ not in open):
                        open.append(adj_)
                        exp_nodes.append(adj_)
                        Len_DFS = len(exp_nodes)
                        if current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500 and ((type ==1) & (Len_DFS>=steps)):
                            print('Random search could not find the path within 10000 steps')
                            check = 1
                            break
                        elif current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500:
                            print('path found')
                            print_path(current_node)
                            #print(current_node)
                            #print("Explored Nodes: ", exp_nodes)
                            print("Length of Explored Nodes DFS: ", Len_DFS)
                            check = 1
                            break
            else:
                if current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500 and ((type ==1) & (Len_DFS>=steps)):
                    print('Random search could not find the path within 10000 steps')
                    check = 1
                    break
                elif current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500:
                    print('path found')
                    print_path(current_node)
                    #print(current_node)
                    Len_DFS = len(exp_nodes)
                    #print("Explored Nodes: ", exp_nodes)
                    print("Length of Explored Nodes DFS: ", Len_DFS)
                    check = 1
                    break
    print('######################### Path found using DFS algorithm #########################################')
    return Len_DFS
######################################  DFS  #############################################
PokerGame_DFSSearch(Root,steps,0)

##################################################################################################################

path found
1
************* state info **************
nn_current_hand 2
nn_current_bidding 0
phase SHOWDOWN
pot 0
acting_agent agent
parent_state <__main__.GameState object at 0x000002653AC12DC0>
children None
agent <__main__.PokerPlayer object at 0x000002653BA09550>
opponent <__main__.PokerPlayer object at 0x000002653BA09580>
---------- showdown ----------
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
['4OfAKind', 'A', 's'] 117
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
['OnePair', '2', 's'] 27
showdown_info agent win
----- agent -----
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
agent.current_hand_type ['4OfAKind', 'A', 's']
agent.current_hand_strength 117
agent.stack 500
agent.action CALL
agent.action_value 5
----- opponent -----
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
opponent.current_hand_type ['OnePair', '2', 's']
opponent.current_hand_strength 27
opponent.stack 300
opponent.action BET
opponent.action_value 25
**************** end ******************

3630

In [3]:
############################### Greedy Search #####################################################################################
def PokerGame_GreedySearch(Root,steps,type):
    open = []
    closed = []
    check = 0
    exp_nodes=[]
    for i in range(len(Root)):
        if check==1:
            break
        node = Root[i]
        open.append(node)
        while len(open) > 0:
            current_node = open.pop(-1)
            closed.append(current_node)
            if current_node.children is not None:
                for adj_ in current_node.children:

                    if (adj_ in closed):
                        continue
                    if (adj_ not in open):
                        if (adj_.nn_current_bidding > current_node.nn_current_bidding) or (adj_.agent.stack > current_node.agent.stack):
                            open.append(adj_)
                            exp_nodes.append(adj_)
                            Len_Greedy = len(exp_nodes)
                    if current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500 and ((type ==1) & (Len_Greedy>=steps)):
                        print('Random search could not find the path within 10000 steps')
                        check=1
                        break
                    elif current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500:
                        print('path found')
                        #print(current_node)
                        print_path(current_node)
                        #print("Explored Nodes: ", exp_nodes)
                        print("Length of Explored Nodes Greedy Search: ", Len_Greedy)
                        check = 1
                        break
            else:
                if current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500 and ((type ==1) & (Len_Greedy>=steps)):
                    print('Random search could not find the path within 10000 steps')
                    check = 1
                    break
                elif current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500:
                    print('path found')
                    print_path(current_node)
                    #print(current_node)
                    Len_Greedy = len(exp_nodes)
                    #print("Explored Nodes: ", exp_nodes)
                    print("Length of Explored Nodes Greedy Search: ", Len_Greedy)
                    check = 1
                    break
    print('######################### Path found using Greedy Seacrh algorithm #########################################')
    return Len_Greedy
########################################### Greedy Search ################################################################

PokerGame_GreedySearch(Root,steps,0)

##############################################################################################################


path found
1
************* state info **************
nn_current_hand 2
nn_current_bidding 0
phase SHOWDOWN
pot 0
acting_agent agent
parent_state <__main__.GameState object at 0x000002653AC12DC0>
children None
agent <__main__.PokerPlayer object at 0x000002653BA09550>
opponent <__main__.PokerPlayer object at 0x000002653BA09580>
---------- showdown ----------
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
['4OfAKind', 'A', 's'] 117
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
['OnePair', '2', 's'] 27
showdown_info agent win
----- agent -----
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
agent.current_hand_type ['4OfAKind', 'A', 's']
agent.current_hand_strength 117
agent.stack 500
agent.action CALL
agent.action_value 5
----- opponent -----
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
opponent.current_hand_type ['OnePair', '2', 's']
opponent.current_hand_strength 27
opponent.stack 300
opponent.action BET
opponent.action_value 25
**************** end ******************

42

In [4]:
######################################### A star #################################################################################
def PokerGame_Astar(Root,steps,type):
    open = []
    closed = []
    count = 0
    exp_nodes=[]
    check=0
    for i in range(len(Root)):
        if check==1:
            break
        node = Root[i]
        open.append(node)
        while len(open) > 0:
            current_node = open.pop(-1)
            closed.append(current_node)
            if current_node.children is not None:
                for adj_ in current_node.children:
                    cost = current_node.agent.stack - adj_.agent.stack
                    if (adj_ in closed):
                        continue
                    if (adj_ not in open):
                        if (cost >= 0):
                            open.append(adj_)
                        elif (cost < 0) and (adj_.nn_current_hand-current_node.nn_current_hand) >0:
                            open.append(adj_)
                            exp_nodes.append(adj_)
                            Len_Astar = len(exp_nodes)
                    if current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500 and ((type ==1) & (Len_Astar>=steps)):
                        print('Random search could not find the path within 10000 steps')
                        check = 1
                        break
                    elif current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500:
                        print('path found')
                        count = count + 1
                        print_path(current_node)
                        #print(current_node)
                        check = 1
                        #print("Explored Nodes: ", exp_nodes)
                        print("Length of Explored Nodes Astar: ", Len_Astar)
                        break
            else:
                if current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500 and ((type ==1) & (Len_Astar>=steps)):
                    print('Random search could not find the path within 10000 steps')
                    check = 1
                    break
                elif current_node.nn_current_hand <= 4 and current_node.phase == 'SHOWDOWN' and current_node.agent.stack >= 500:
                    print('path found')
                    count = count + 1
                    print_path(current_node)
                    #print(current_node)
                    check = 1
                    #print("Explored Nodes: ", exp_nodes)
                    Len_Astar=len(exp_nodes)
                    print("Length of Explored Nodes Astar: ", Len_Astar)
                    break

    print('######################### Path found using A star algorithm #########################################')
    return Len_Astar
################################################ Astar ###################################################

PokerGame_Astar(Root,steps,0)

##############################################

path found
1
************* state info **************
nn_current_hand 2
nn_current_bidding 0
phase SHOWDOWN
pot 0
acting_agent agent
parent_state <__main__.GameState object at 0x000002653AC12DC0>
children None
agent <__main__.PokerPlayer object at 0x000002653BA09550>
opponent <__main__.PokerPlayer object at 0x000002653BA09580>
---------- showdown ----------
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
['4OfAKind', 'A', 's'] 117
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
['OnePair', '2', 's'] 27
showdown_info agent win
----- agent -----
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
agent.current_hand_type ['4OfAKind', 'A', 's']
agent.current_hand_strength 117
agent.stack 500
agent.action CALL
agent.action_value 5
----- opponent -----
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
opponent.current_hand_type ['OnePair', '2', 's']
opponent.current_hand_strength 27
opponent.stack 300
opponent.action BET
opponent.action_value 25
**************** end ******************

373

In [5]:
################################# Random Search ############################################################

def PokerGame_Random(Root,steps,type):
    select = random.randint(1, 3)
    print(select)
    if (select == 1):
        typeofSearch = "DFS"
        print("Random search found DFS")
        PokerGame_DFSSearch(Root,steps,1)
    elif (select == 2):
        typeofSearch = "Greedy"
        print("Random search found Greedy")
        PokerGame_GreedySearch(Root,steps,1)
    elif (select == 3):
        typeofSearch = "aStar"
        print("Random search found aStarE")
        PokerGame_Astar(Root,steps,1)

###############################################################################################################

PokerGame_Random(Root,steps,1)

############################################################################################################################

3
Random search found aStarE
path found
1
************* state info **************
nn_current_hand 2
nn_current_bidding 0
phase SHOWDOWN
pot 0
acting_agent agent
parent_state <__main__.GameState object at 0x000002653AC12DC0>
children None
agent <__main__.PokerPlayer object at 0x000002653BA09550>
opponent <__main__.PokerPlayer object at 0x000002653BA09580>
---------- showdown ----------
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
['4OfAKind', 'A', 's'] 117
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
['OnePair', '2', 's'] 27
showdown_info agent win
----- agent -----
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
agent.current_hand_type ['4OfAKind', 'A', 's']
agent.current_hand_strength 117
agent.stack 500
agent.action CALL
agent.action_value 5
----- opponent -----
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
opponent.current_hand_type ['OnePair', '2', 's']
opponent.current_hand_strength 27
opponent.stack 300
opponent.action BET
opponent.action_value 25
**********

In [6]:
# print all states

state__ = end_state_
nn_level = 0

print('------------ print game info ---------------')
print('nn_states_total', len(game_state_queue))

while state__.parent_state != None:
    nn_level += 1
    print(nn_level)
    state__.print_state_info()
    state__ = state__.parent_state

print(nn_level)



------------ print game info ---------------
nn_states_total 61658
1
************* state info **************
nn_current_hand 2
nn_current_bidding 0
phase SHOWDOWN
pot 0
acting_agent agent
parent_state <__main__.GameState object at 0x000002653AB0DC40>
children None
agent <__main__.PokerPlayer object at 0x000002653B7E7580>
opponent <__main__.PokerPlayer object at 0x000002653B7E75B0>
---------- showdown ----------
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
['4OfAKind', 'A', 's'] 117
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
['OnePair', '2', 's'] 27
showdown_info agent win
----- agent -----
agent.current_hand ['As', 'Ac', '3s', 'Ad', 'Ah']
agent.current_hand_type ['4OfAKind', 'A', 's']
agent.current_hand_strength 117
agent.stack 500
agent.action CALL
agent.action_value 5
----- opponent -----
opponent.current_hand ['Kd', '7d', '2s', '5h', '2c']
opponent.current_hand_type ['OnePair', '2', 's']
opponent.current_hand_strength 27
opponent.stack 300
opponent.action BET
opponent

In [7]:
# perform your searching algorithm